# Split dataset into `train`/`test` folders

Using split-folders package: https://pypi.org/project/split-folders/

In [1]:
import splitfolders

From instructions on package's website:

In [6]:
# # Split with a ratio.
# # To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
# splitfolders.ratio("raw_images", 
#                    output="input_images", 
#                    seed=42, 
#                    ratio=(.8, .2))

**Tues 10/12, 1:30pm:**

It worked! The package created copies of all my files, sorted into `train` and `val` folders, each containing the originally named folders/split. 

For now I'm going to delete these, then re-run this code after I have added to the dataset a little today.

In [7]:
# Split with a fixed number of items.
# To only split into training and validation set, use a single number to `fixed`, i.e., `10`.
splitfolders.fixed("raw_images", 
                   output="input_images", 
                   seed=42, 
                   fixed=(100))

Copying files: 1461 files [00:15, 91.90 files/s] 


**Wed 10/13, 10:30am:**

Performed operation on full dataset after cropping images.

Decided to set aside 100 images of each class as holdout/test set and will use `validation_split` parameter in Keras's `ImageDataGenerator` to split the training set during model training.